In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os
import numpy as np
import pickle
import json
import pandas as pd
from datetime import datetime
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
from nilearn.plotting import plot_design_matrix
from nilearn.image import load_img
from nilearn.reporting import make_glm_report
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
sys.path.append('..')
from utils.data import Subject, load_participant_list, create_dummy_regressors
from utils.analysis import compute_parametric_modulator

In [2]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = "/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-24.0.1"

sub_ids = load_participant_list(base_dir)

In [3]:
#subjects = [Subject(base_dir, sub_id, include_modeling=True, include_imaging=True, bids_dir=bids_dir) for sub_id in sub_ids]
subject = Subject(base_dir, '02', include_modeling=True, include_imaging=True, bids_dir=bids_dir)

In [17]:
model_params = {
    'model_name': 'desperate',
    'tr': 2.33384,
    'hrf_model': 'spm',
    'noise_model': 'ar1',
    'smoothing_fwhm': 5,
    'high_pass': 0.01,
    'motion_type': 'basic',
    'fd_thresh': 0.5,
    'std_dvars_thresh': 2.5,
    'scrub': 'dummies',
    'modulators': 'both',
    'exclude_stimuli': True,
    'include_physio': True,
    'brain_mask': True,
    'modulator_normalization': 'zscore'
}

run = 'learning1'

In [18]:
# Parameters
model_name = model_params["model_name"]
tr = model_params["tr"]
hrf_model = model_params["hrf_model"]
noise_model = model_params["noise_model"]
smoothing_fwhm = model_params["smoothing_fwhm"]
high_pass = model_params["high_pass"]
include_physio = model_params["include_physio"]
brain_mask = model_params["brain_mask"]
modulator_normalization = model_params["modulator_normalization"]
modulators = model_params["modulators"]
exclude_stimuli = model_params["exclude_stimuli"]
motion_type = model_params["motion_type"]
fd_thresh = model_params["fd_thresh"]
std_dvars_thresh = model_params["std_dvars_thresh"]
scrub = model_params["scrub"]

In [19]:
# Load confounds
confounds, sample_mask = subject.load_confounds(run, motion_type=motion_type,
                                                fd_thresh=fd_thresh, std_dvars_thresh=std_dvars_thresh,
                                                scrub=(0 if scrub == 'dummies' else scrub))
if include_physio:
    physio_regressors = subject.load_physio_regressors(run)
    confounds = confounds.join(physio_regressors)

In [20]:
if scrub == 'dummies':
    dummies = create_dummy_regressors(sample_mask, len(confounds))
    confounds = pd.concat([confounds, dummies], axis=1)

In [21]:
# Load fMRI volume
img_path = subject.img.get(run)
fmri_img = load_img(img_path)

In [22]:
if brain_mask:
    brain_mask_path = subject.brain_mask.get(run)
    brain_mask = load_img(brain_mask_path)
else:
    brain_mask = None

n = fmri_img.shape[-1]
frametimes = np.linspace(tr / 2., (n - .5) * tr, n)

In [23]:
if modulators == 'both':
    columns_event = {'first_stim_value_rl':'first_stim_presentation',
                    'first_stim_value_ck':'first_stim_presentation',
                    'first_stim':'first_stim_presentation'}
    # Load events
    events = getattr(subject, run).extend_events_df(columns_event)
elif modulators == 'none':
    events = getattr(subject, run).events
else:
    raise NotImplementedError("Modulator type not implemented")

# Collapsing events

In [24]:
events.head(20)

,onset,duration,trial_type,trial,first_stim_value_rl,first_stim_value_ck,first_stim
0,0.005624,0.817359,first_stim_presentation,1,2.332217,0.000000,7.0
1,0.822983,0.615436,second_stim_presentation,1,0.000000,0.000000,0.0
2,1.438419,0.000000,response,1,0.000000,0.000000,0.0
3,1.456892,1.918454,purple_frame,1,0.000000,0.000000,0.0
4,1.974071,1.401274,points_feedback,1,0.000000,0.000000,0.0
5,3.375345,7.957434,iti,1,0.000000,0.000000,0.0
6,11.332779,0.834021,first_stim_presentation,2,4.000000,0.003004,7.0
7,12.166800,0.478543,second_stim_presentation,2,0.000000,0.000000,0.0
8,12.645344,0.000000,response,2,0.000000,0.000000,0.0
9,12.650588,2.051938,purple_frame,2,0.000000,0.000000,0.0


In [15]:
def merge_response_and_purple_frame(trial_df):
    # Only merge if both 'response' and 'purple_frame' exist in this trial
    if {'response', 'purple_frame'}.issubset(trial_df['trial_type'].unique()):
        # Grab the first row of each type
        resp = trial_df.loc[trial_df['trial_type'] == 'response'].iloc[0]
        purple = trial_df.loc[trial_df['trial_type'] == 'purple_frame'].iloc[0]

        # Compute new onset/duration
        new_onset = resp['onset']
        new_duration = (purple['onset'] + purple['duration']) - new_onset
        
        # Make a copy of the response row to mutate into a merged event
        merged = resp.copy()
        merged['trial_type'] = 'response_purple_frame'
        merged['onset'] = new_onset
        merged['duration'] = new_duration
        
        # Drop the old rows, then append the merged row
        trial_df = trial_df[~trial_df['trial_type'].isin(['response', 'purple_frame'])]
        return pd.concat([trial_df, pd.DataFrame([merged])], ignore_index=True)
    else:
        # Nothing to merge, just return the original group
        return trial_df

df_merged = (
    df.groupby('trial', group_keys=False)
      .apply(merge_response_and_purple_frame)
      .sort_values(['trial', 'onset'])
      .reset_index(drop=True)
)

In [16]:
collapse_events(events).head(20)

,trial_type,onset,duration
0,first_stim_presentation,0.009226,119.750247
1,iti,3.362241,912.541368
2,purple_frame,1.210246,280.026850
3,response,1.191453,0.000000
4,second_stim_presentation,0.826583,61.251577


In [12]:
if exclude_stimuli:
    events['trial_type'] = events.apply(
        lambda row: f"{row['trial_type']}_{'exclude' if int(row['first_stim']) in (1, 8) else 'include'}"
        if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
        axis=1
    ) 

In [13]:
# Ignore warnings related to null duration events and unexpected columns in events data
warnings.filterwarnings("ignore", message=".*events with null duration.*")
warnings.filterwarnings("ignore", message=".*following unexpected columns in events data.*")
# Create design matrix
design_matrix = make_first_level_design_matrix(frame_times=frametimes,
                                    events=events,
                                    hrf_model=hrf_model,
                                    drift_model=None,
                                    high_pass=high_pass,
                                    add_regs=confounds)

In [14]:
if modulators == 'both':
    # RL Parametric modulation
    parametric_modulator_column = 'first_stim_value_rl'
    condition = 'first_stim_presentation_include' if exclude_stimuli else 'first_stim_presentation'
    reg_value = compute_parametric_modulator(events, condition, parametric_modulator_column,
                                                frametimes, hrf_model, normalize=modulator_normalization)
    design_matrix.insert(1, parametric_modulator_column, reg_value)

    # CK Parametric modulation
    parametric_modulator_column = 'first_stim_value_ck'
    condition = 'first_stim_presentation_include' if exclude_stimuli else 'first_stim_presentation'
    reg_value = compute_parametric_modulator(events, condition, parametric_modulator_column,
                                                frametimes, hrf_model, normalize=modulator_normalization)
    design_matrix.insert(2, parametric_modulator_column, reg_value)

In [15]:
# Compute the average correlation matrix
# average_correlation_matrix = design_matrix.corr().values
# plt.figure(figsize=(30, 28))
# sns.heatmap(average_correlation_matrix, xticklabels=design_matrix.columns, 
#             yticklabels=design_matrix.columns, cmap='coolwarm', annot=True)
# plt.title('Average Correlation Matrix')
# plt.show()